# Введение в MapReduce модель на Python


In [1]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [2]:
def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [3]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [4]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [5]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [6]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [7]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [8]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [9]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [10]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [11]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [12]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [13]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [14]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [15]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 1.275448460722387),
 (1, 1.275448460722387),
 (2, 1.275448460722387),
 (3, 1.275448460722387),
 (4, 1.275448460722387)]

## Inverted index

In [16]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('it', ['0', '1', '2']),
 ('what', ['0', '1']),
 ('is', ['0', '1', '2']),
 ('a', ['2']),
 ('banana', ['2'])]

## WordCount

In [17]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [18]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [19]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('banana', 2), ('what', 10)]),
 (1, [('is', 18), ('it', 18)])]

## TeraSort

In [20]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.005981062479298305),
   (None, 0.015761060546713335),
   (None, 0.03679871171019211),
   (None, 0.052437095405092116),
   (None, 0.05342025172191778),
   (None, 0.16107894959885172),
   (None, 0.16949899499029353),
   (None, 0.1700782116909365),
   (None, 0.2655482668904836),
   (None, 0.29345425665843683),
   (None, 0.31639675840487935),
   (None, 0.3180100082737155),
   (None, 0.34890515291192803),
   (None, 0.41662541407306886),
   (None, 0.4360861646801152)]),
 (1,
  [(None, 0.5153151120884983),
   (None, 0.5358068711160389),
   (None, 0.5538814289357271),
   (None, 0.5892307240265636),
   (None, 0.608701382511187),
   (None, 0.6425117029301601),
   (None, 0.6788228033594661),
   (None, 0.6934341459646108),
   (None, 0.7031184165410632),
   (None, 0.7042351438619909),
   (None, 0.7899975698075133),
   (None, 0.8271080923156376),
   (None, 0.8661641414373076),
   (None, 0.8923793379043262),
   (None, 0.9330326072968295)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [21]:
import random
from functools import reduce

def RECORDREADER(count, part_size):
    data = [random.randint(-100, 100) for _ in range(count)]
    for i in range(0, count, part_size):
        yield data[i:i+part_size]

def MAP(num_list):
    return max(num_list)

def REDUCE(max1, max2):
    return max(max1, max2)

num_count = 20
part_size = 5

record_reader_list = list(RECORDREADER(num_count, part_size))

full_list_nums = [num for part in record_reader_list for num in part]
print("Исходный список чисел:", full_list_nums)
print("Разбитый на части список:", record_reader_list)

mapped_values = list(map(MAP, record_reader_list))
print("Локальные максимумы:", mapped_values)

result = reduce(REDUCE, mapped_values)
print("Глобальный максимум:", result)

Исходный список чисел: [27, -40, 5, 97, 89, -20, -11, -5, 1, 0, 29, -77, 24, -70, -40, 3, 89, 74, 52, -79]
Разбитый на части список: [[27, -40, 5, 97, 89], [-20, -11, -5, 1, 0], [29, -77, 24, -70, -40], [3, 89, 74, 52, -79]]
Локальные максимумы: [97, 1, 29, 89]
Глобальный максимум: 97


### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [22]:
import random
from functools import reduce

def RECORDREADER(count, part_size):
    data = [random.randint(0, 100) for _ in range(count)]
    for i in range(0, count, part_size):
        yield data[i:i+part_size]

def MAP(num_list):
    return (sum(num_list), len(num_list))

def REDUCE(accum, value):
    total_sum = accum[0] + value[0]
    total_count = accum[1] + value[1]
    return (total_sum, total_count)

num_count = 20
part_size = 5

record_reader_list = list(RECORDREADER(num_count, part_size))

full_list_nums = [num for part in record_reader_list for num in part]
print("Исходный список чисел:", full_list_nums)
print("Разбитый на части список:", record_reader_list)

mapped_values = list(map(MAP, record_reader_list))
print("Локальные суммы и количества элементов:", mapped_values)

total_sum, total_count = reduce(REDUCE, mapped_values)
print("Арифметическое среднее:", total_sum / total_count)

Исходный список чисел: [99, 12, 98, 23, 50, 82, 72, 72, 22, 31, 37, 45, 4, 22, 52, 85, 41, 9, 3, 43]
Разбитый на части список: [[99, 12, 98, 23, 50], [82, 72, 72, 22, 31], [37, 45, 4, 22, 52], [85, 41, 9, 3, 43]]
Локальные суммы и количества элементов: [(282, 5), (279, 5), (160, 5), (181, 5)]
Арифметическое среднее: 45.1


### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [23]:
import random
from functools import reduce

def RECORDREADER(count):
    return [random.randint(0, 10) for _ in range(count)]

def MAP(num):
    key = num % 3
    return key, (num, 1)

def REDUCE(a, b):
    total_sum = a[0] + b[0]
    total_count = a[1] + b[1]
    return total_sum, total_count

def groupByKey(mapped_data):
    mapped_data.sort(key=lambda x: x[0])
    grouped_data = {}
    for key, value in mapped_data:
        grouped_data.setdefault(key, []).append(value)
    return grouped_data.items()

data = RECORDREADER(20)
print("Исходные данные:", data)

map_list = list(map(MAP, data))
print("\nРезультат MAP:")
for value in map_list:
    print(value)

grouped_data = groupByKey(map_list)
print("\nГруппированные данные:")
for key, values in grouped_data:
    print(f"Ключ {key}: {values}")

reduced_values = [(key, reduce(REDUCE, values)) for key, values in grouped_data]

group_result = {key: total_sum / total_count for key, (total_sum, total_count) in reduced_values}

print("\nАрифметическое среднее в каждой группе:")
for key, value in group_result.items():
    print(f"Ключ {key}: {value}")

total_sum, total_count = reduce(REDUCE, [values for _, values in reduced_values])
print("\nОбщее арифметическое среднее:", total_sum / total_count)

Исходные данные: [4, 1, 9, 4, 9, 7, 9, 10, 3, 8, 9, 5, 9, 6, 0, 9, 7, 3, 5, 9]

Результат MAP:
(1, (4, 1))
(1, (1, 1))
(0, (9, 1))
(1, (4, 1))
(0, (9, 1))
(1, (7, 1))
(0, (9, 1))
(1, (10, 1))
(0, (3, 1))
(2, (8, 1))
(0, (9, 1))
(2, (5, 1))
(0, (9, 1))
(0, (6, 1))
(0, (0, 1))
(0, (9, 1))
(1, (7, 1))
(0, (3, 1))
(2, (5, 1))
(0, (9, 1))

Группированные данные:
Ключ 0: [(9, 1), (9, 1), (9, 1), (3, 1), (9, 1), (9, 1), (6, 1), (0, 1), (9, 1), (3, 1), (9, 1)]
Ключ 1: [(4, 1), (1, 1), (4, 1), (7, 1), (10, 1), (7, 1)]
Ключ 2: [(8, 1), (5, 1), (5, 1)]

Арифметическое среднее в каждой группе:
Ключ 0: 6.818181818181818
Ключ 1: 5.5
Ключ 2: 6.0

Общее арифметическое среднее: 6.3


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [24]:
from typing import Iterator
import numpy as np

def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def groupbykey(iterable):
    grouped_data = {}
    for key, value in iterable:
        grouped_data.setdefault(key, []).append(value)
    return list(grouped_data.items())

def groupbykey_distributed(map_partitions, PARTITIONER):
    global reducers
    partitions = [dict() for _ in range(reducers)]
    for map_partition in map_partitions:
        for key, value in map_partition:
            p = partitions[PARTITIONER(key)]
            p.setdefault(key, []).append(value)
    return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
    global reducers
    return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
    map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

    if COMBINER is not None:
        map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

    reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)

    reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group),
                                                                                      reduce_partition[1]))), reduce_partitions)
    total_values_sent = sum([len(vs) for (k, vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])

    print(f"{total_values_sent} key-value pairs were sent over a network.")
    return reduce_outputs

documents = [
    "apple banana orange apple",
    "banana apple grape",
    "grape banana orange",
    "apple orange banana apple",
]

maps = 2
reducers = 2

def INPUTFORMAT():
    global maps

    def RECORDREADER(split):
        for (docid, document) in enumerate(split):
            for word in document.split(" "):
                yield (docid, word)

    split_size = int(np.ceil(len(documents) / maps))
    for i in range(0, len(documents), split_size):
        yield RECORDREADER(documents[i:i + split_size])

def MAP(docId: int, word: str):
    yield (word, None)

def REDUCE(key: str, _):
    yield key

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(partitioned_output)

14 key-value pairs were sent over a network.
[(0, ['banana', 'grape', 'orange']), (1, ['apple'])]


#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [25]:
import random

def RECORDREADER(count):
  data = []
  for _ in range(count):
    data.append((random.randint(0, 100), random.randint(0, 100)))
  return data

def MAP(list_tuples):
  return [(t, t) for t in list_tuples if C(t)]

def REDUCE(map_list):
  return [t for t, _ in map_list]

def C(t):
  return t[0] % 2 == 0 and t[1] % 2 == 0

record = RECORDREADER(10)

mapped = MAP(record)
reduced = REDUCE(mapped)

print("Исходные данные:", record)
print("MAP:", mapped)
print("REDUCE:", reduced)

Исходные данные: [(7, 80), (72, 0), (84, 78), (72, 3), (57, 33), (81, 43), (57, 13), (46, 17), (9, 40), (81, 32)]
MAP: [((72, 0), (72, 0)), ((84, 78), (84, 78))]
REDUCE: [(72, 0), (84, 78)]


### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [27]:
import random

def RECORDREADER(count):
  data = []
  for _ in range(count):
    data.append((random.randint(0, 100), random.randint(0, 100)))
  return data

def MAP(t):
  S = {10, 20, 30, 40, 50}
  res = tuple(el for el in t if el in S)
  return (res, res) if res else None

def REDUCE(key, values):
  return (key, key)


def group_by_key(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

record = RECORDREADER(30)
print("RECORDREADER:", record)

mapped = [MAP(x) for x in record if MAP(x) is not None]
print("MAP:", mapped)

shuffle_output = list(group_by_key(mapped))
print("Shuffle:", shuffle_output)

reduced = [REDUCE(k, v) for k, v in shuffle_output]
print("REDUCE:", reduced)

RECORDREADER: [(97, 48), (61, 72), (96, 37), (18, 76), (90, 67), (76, 50), (47, 27), (65, 19), (65, 54), (83, 37), (49, 35), (59, 29), (12, 33), (81, 59), (1, 88), (18, 30), (12, 16), (63, 72), (1, 85), (28, 42), (69, 97), (71, 53), (26, 83), (12, 87), (14, 44), (56, 44), (99, 78), (19, 71), (71, 17), (19, 80)]
MAP: [((50,), (50,)), ((30,), (30,))]
Shuffle: [((50,), [(50,)]), ((30,), [(30,)])]
REDUCE: [((50,), (50,)), ((30,), (30,))]


### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [28]:
import random

def RECORDREADER(count):
    return [(random.randint(0, 100), random.randint(0, 100)) for _ in range(count)]

def MAP(t):
    return (t, t)

def REDUCE(key, values):
    return (key, key)

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

record = RECORDREADER(15)
print("RECORDREADER:", record)

map_output = [MAP(x) for x in record]
print("MAP output:", map_output)

shuffle_output = list(group_by_key(map_output))
print("Shuffle output:", shuffle_output)

reduce_output = [REDUCE(k, v) for k, v in shuffle_output]
print("Reduce output:", reduce_output)

RECORDREADER: [(87, 62), (94, 71), (96, 98), (43, 48), (88, 9), (4, 88), (23, 93), (62, 94), (40, 66), (3, 54), (79, 97), (37, 55), (88, 26), (90, 26), (70, 23)]
MAP output: [((87, 62), (87, 62)), ((94, 71), (94, 71)), ((96, 98), (96, 98)), ((43, 48), (43, 48)), ((88, 9), (88, 9)), ((4, 88), (4, 88)), ((23, 93), (23, 93)), ((62, 94), (62, 94)), ((40, 66), (40, 66)), ((3, 54), (3, 54)), ((79, 97), (79, 97)), ((37, 55), (37, 55)), ((88, 26), (88, 26)), ((90, 26), (90, 26)), ((70, 23), (70, 23))]
Shuffle output: [((87, 62), [(87, 62)]), ((94, 71), [(94, 71)]), ((96, 98), [(96, 98)]), ((43, 48), [(43, 48)]), ((88, 9), [(88, 9)]), ((4, 88), [(4, 88)]), ((23, 93), [(23, 93)]), ((62, 94), [(62, 94)]), ((40, 66), [(40, 66)]), ((3, 54), [(3, 54)]), ((79, 97), [(79, 97)]), ((37, 55), [(37, 55)]), ((88, 26), [(88, 26)]), ((90, 26), [(90, 26)]), ((70, 23), [(70, 23)])]
Reduce output: [((87, 62), (87, 62)), ((94, 71), (94, 71)), ((96, 98), (96, 98)), ((43, 48), (43, 48)), ((88, 9), (88, 9)), ((4, 8

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [29]:
import random

def RECORDREADER(count):
    return [(random.randint(0, 5), random.randint(0, 5)) for _ in range(count)]

def MAP(t):
    return (t, t)

def REDUCE(key, values):
    if len(values) == 2:
        return (key, key)

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

record = RECORDREADER(50)
print("RECORDREADER:", record)

map_output = [MAP(x) for x in record]
print("MAP output:", map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)
print("Shuffle output:", shuffle_output)

reduce_output = [val for val in map(lambda x: REDUCE(*x), shuffle_output) if val is not None]
print("Reduce output:", reduce_output)

RECORDREADER: [(0, 3), (3, 4), (0, 3), (3, 3), (2, 3), (2, 3), (3, 0), (1, 0), (3, 3), (2, 0), (1, 0), (4, 1), (4, 3), (1, 2), (4, 4), (3, 2), (2, 0), (2, 0), (0, 5), (3, 4), (0, 0), (3, 4), (2, 4), (4, 0), (3, 4), (0, 0), (3, 3), (3, 3), (4, 5), (5, 3), (4, 4), (1, 3), (5, 2), (3, 1), (1, 1), (2, 5), (4, 3), (1, 0), (1, 5), (5, 1), (3, 1), (0, 3), (0, 3), (4, 5), (3, 5), (1, 0), (4, 2), (5, 0), (0, 5), (5, 0)]
MAP output: [((0, 3), (0, 3)), ((3, 4), (3, 4)), ((0, 3), (0, 3)), ((3, 3), (3, 3)), ((2, 3), (2, 3)), ((2, 3), (2, 3)), ((3, 0), (3, 0)), ((1, 0), (1, 0)), ((3, 3), (3, 3)), ((2, 0), (2, 0)), ((1, 0), (1, 0)), ((4, 1), (4, 1)), ((4, 3), (4, 3)), ((1, 2), (1, 2)), ((4, 4), (4, 4)), ((3, 2), (3, 2)), ((2, 0), (2, 0)), ((2, 0), (2, 0)), ((0, 5), (0, 5)), ((3, 4), (3, 4)), ((0, 0), (0, 0)), ((3, 4), (3, 4)), ((2, 4), (2, 4)), ((4, 0), (4, 0)), ((3, 4), (3, 4)), ((0, 0), (0, 0)), ((3, 3), (3, 3)), ((3, 3), (3, 3)), ((4, 5), (4, 5)), ((5, 3), (5, 3)), ((4, 4), (4, 4)), ((1, 3), (1, 3

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [30]:
import random

rels = [1, 2]

class Tuple:
    def __init__(self, data: tuple, rel_id: int):
        self.data = data
        self.rel_id = rel_id

def get_random_tuple(count):
    data = tuple([(random.randint(0, 5), random.randint(0, 5)) for i in range(count)])
    rel_id = rels[random.randint(0, len(rels) - 1)]
    return Tuple(data, rel_id)

def RECORDREADER(count):
    return [get_random_tuple(4) for i in range(count)]

def MAP(t: Tuple):
    return (t.data, t.rel_id)

def REDUCE(key, values):
    if values == [rels[0]]:
        return (key, key)

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

record = RECORDREADER(50)

map_output = list(map(lambda x: MAP(x), RECORDREADER(50)))
print("MAP output:", map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)
print("Shuffle output:", shuffle_output)

reduce_output = [el[0] for el in list(map(lambda x: REDUCE(*x), shuffle_output)) if el is not None]
print("Reduce output:", reduce_output)

MAP output: [(((2, 3), (4, 1), (5, 5), (3, 3)), 2), (((4, 2), (3, 5), (0, 3), (1, 4)), 1), (((1, 4), (4, 4), (2, 3), (5, 4)), 2), (((5, 3), (3, 0), (5, 2), (2, 5)), 1), (((4, 5), (1, 3), (0, 1), (4, 0)), 2), (((2, 3), (0, 0), (1, 4), (2, 0)), 2), (((0, 0), (4, 5), (0, 5), (1, 5)), 2), (((0, 1), (1, 3), (5, 3), (3, 4)), 1), (((5, 0), (5, 3), (5, 4), (5, 4)), 2), (((3, 5), (0, 5), (0, 1), (4, 4)), 1), (((0, 3), (4, 5), (5, 0), (1, 4)), 1), (((5, 1), (5, 0), (0, 4), (5, 4)), 1), (((0, 3), (0, 4), (4, 5), (4, 5)), 2), (((1, 4), (3, 2), (3, 1), (3, 0)), 1), (((0, 5), (2, 2), (5, 1), (0, 4)), 1), (((2, 1), (4, 0), (1, 1), (1, 3)), 2), (((4, 0), (0, 0), (3, 0), (2, 2)), 1), (((3, 5), (1, 4), (5, 2), (1, 5)), 1), (((4, 2), (1, 0), (1, 5), (3, 2)), 2), (((3, 1), (3, 5), (0, 2), (0, 4)), 1), (((1, 2), (2, 0), (4, 5), (5, 3)), 2), (((1, 4), (0, 4), (3, 1), (3, 1)), 2), (((0, 4), (5, 1), (3, 0), (4, 1)), 1), (((3, 4), (4, 1), (4, 3), (0, 5)), 2), (((2, 4), (0, 1), (0, 1), (5, 1)), 1), (((5, 5), (0

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [31]:
import random

rels = [1, 2]

class Tuple:
    def __init__(self, data: tuple, rel_id: int):
        self.data = data
        self.rel_id = rel_id

def get_random_tuple():
    data = (random.randint(0, 4), random.randint(0, 4))
    rel_id = rels[random.randint(0, len(rels) - 1)]
    return Tuple(data, rel_id)

def RECORDREADER(count):
    return [get_random_tuple() for i in range(count)]

def MAP(t: Tuple):
    if t.rel_id == rels[0]:
        return (t.data[1], (t.rel_id, t.data[0]))
    else:
        return (t.data[0], (t.rel_id, t.data[1]))

def REDUCE(key, values):
    res = []
    for v in values:
        res.append((v[0], key, v[1]))
    return res

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

record = RECORDREADER(80)

map_output = list(map(lambda x: MAP(x), RECORDREADER(80)))
print("MAP output:", map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)
print("Shuffle output:", shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x), shuffle_output))
print("Reduce output:", reduce_output)

MAP output: [(2, (1, 4)), (0, (2, 2)), (1, (1, 1)), (1, (2, 2)), (2, (1, 3)), (3, (2, 4)), (0, (2, 1)), (1, (1, 2)), (4, (1, 4)), (3, (2, 2)), (2, (1, 0)), (3, (1, 1)), (3, (1, 0)), (2, (1, 0)), (0, (2, 3)), (0, (2, 0)), (3, (2, 1)), (1, (1, 3)), (4, (1, 0)), (1, (1, 3)), (1, (2, 2)), (3, (1, 2)), (3, (1, 0)), (0, (1, 4)), (4, (2, 2)), (2, (1, 4)), (3, (1, 4)), (4, (1, 0)), (4, (2, 0)), (4, (1, 0)), (2, (2, 3)), (4, (2, 3)), (4, (2, 3)), (2, (2, 3)), (2, (2, 4)), (1, (2, 1)), (4, (1, 2)), (4, (2, 4)), (3, (1, 0)), (4, (2, 1)), (1, (2, 1)), (0, (2, 0)), (0, (1, 3)), (0, (1, 0)), (0, (2, 0)), (4, (2, 0)), (4, (2, 2)), (3, (2, 2)), (3, (2, 3)), (2, (2, 2)), (1, (1, 0)), (1, (2, 2)), (4, (1, 1)), (2, (1, 3)), (0, (1, 0)), (2, (2, 1)), (0, (1, 2)), (3, (2, 2)), (3, (1, 0)), (3, (1, 4)), (2, (2, 1)), (3, (2, 3)), (4, (2, 3)), (3, (1, 0)), (3, (2, 2)), (2, (1, 2)), (0, (2, 2)), (1, (1, 0)), (4, (2, 4)), (1, (1, 3)), (4, (1, 4)), (2, (2, 2)), (4, (2, 2)), (1, (1, 0)), (2, (2, 0)), (0, (2, 0)),

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [33]:
def get_random_tuple():
    return (random.randint(0, 3), random.randint(0, 3), random.randint(0, 3))

def RECORDREADER(count):
    return [get_random_tuple() for i in range(count)]

def MAP(t):
    return (t[0], t[1])

def REDUCE(key, values):
    return (key, max(values))

def group_by_key(iterable):
    t = {}
    for (k2, v2) in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

record = RECORDREADER(100)

map_output = list(map(lambda x: MAP(x), RECORDREADER(100)))
print("MAP output:", map_output)

shuffle_output = group_by_key(map_output)
shuffle_output = list(shuffle_output)
print("Shuffle output:", shuffle_output)

reduce_output = list(map(lambda x: REDUCE(*x), shuffle_output))
print("Reduce output:", reduce_output)

MAP output: [(0, 3), (2, 1), (3, 1), (1, 3), (1, 2), (1, 3), (0, 2), (1, 2), (1, 0), (1, 2), (2, 1), (3, 3), (1, 0), (3, 1), (1, 0), (2, 0), (0, 2), (3, 1), (2, 2), (1, 3), (1, 0), (3, 2), (2, 0), (0, 3), (1, 2), (1, 0), (0, 3), (1, 3), (0, 1), (0, 0), (2, 0), (1, 2), (2, 1), (2, 2), (3, 2), (3, 2), (2, 1), (2, 3), (3, 1), (0, 0), (3, 1), (0, 1), (3, 3), (1, 3), (2, 0), (2, 1), (0, 0), (2, 1), (0, 1), (1, 1), (1, 2), (3, 3), (3, 3), (2, 1), (0, 1), (2, 1), (3, 1), (2, 1), (3, 3), (1, 2), (0, 0), (0, 1), (2, 2), (3, 2), (1, 0), (0, 0), (3, 3), (1, 0), (1, 2), (3, 0), (0, 3), (0, 3), (2, 3), (1, 2), (0, 2), (0, 2), (1, 2), (1, 3), (3, 0), (1, 2), (0, 1), (0, 2), (3, 2), (0, 0), (3, 3), (3, 1), (2, 1), (2, 3), (0, 0), (2, 2), (1, 1), (2, 3), (1, 0), (2, 2), (1, 2), (1, 3), (1, 2), (0, 0), (1, 2), (3, 2)]
Shuffle output: [(0, [3, 2, 2, 3, 3, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 3, 3, 2, 2, 1, 2, 0, 0, 0]), (2, [1, 1, 0, 2, 0, 0, 1, 2, 1, 3, 0, 1, 1, 1, 1, 1, 2, 3, 1, 3, 2, 3, 2]), (3, [1, 3, 1, 1

#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [34]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [38]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  for i in range(small_mat.shape[0]):
    yield ((i, k), w * small_mat[i][j])

def REDUCE(key, values):
  (i, k) = key
  el_value = 0
  for v in values:
    el_value += v
  yield ((i, k), el_value)

Проверьте своё решение

In [39]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [40]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [42]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J)
big_mat = np.random.rand(J,K)
reference_solution = np.matmul(small_mat, big_mat)

def RECORDREADER():
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      yield ((0, i, j), small_mat[i,j])

  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((1, j, k), big_mat[j,k])

def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))

def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])


def MAP_MUL(k1, v1):
  (i, k) = k1
  yield ((i, k), v1)

def REDUCE_MUL(key, values):
  res_el_value = 0
  for v in values:
    res_el_value += v
  yield (key, res_el_value)

def GET_JOINED():
  for j in joined:
    yield j


joined = MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN)
solution = MapReduce(GET_JOINED, MAP_MUL, REDUCE_MUL)
np.allclose(reference_solution, asmatrix(solution))

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [43]:
I = 2
J = 3
K = 4*10

small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

reference_solution = np.matmul(small_mat, big_mat)

def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)

  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output) + 1
  K = max(k for ((i,k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

def INPUTFORMAT():
  first_mat = []
  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j]))
  yield first_mat

  second_mat = []
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k]))
  yield second_mat

def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1
  if mat_num == 0:
    yield (j, (mat_num, i, w))
  else:
    yield (i, (mat_num, j, w))

def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]
  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])

def GET_JOINED():
  for j in joined:
    print("aa", j)
    yield j[1]

def MAP_MUL(k1, v1):
  yield (k1, v1)

def REDUCE_MUL(key, values):
  res_val = 0
  for v in values:
    res_val += v
  yield (key, res_val)

maps = 4
reducers = 2

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []
for p in pre_result:
  for v in p[1]:
    solution.append(v)

print(solution)

np.allclose(reference_solution, asmatrix(solution))

126 key-value pairs were sent over a network.
[(0, [((0, 0), 0.022497384233530725), ((0, 1), 0.0873459877131581), ((0, 2), 0.10457130977895128), ((0, 3), 0.07195234429218929), ((0, 4), 0.06426871423075577), ((0, 5), 0.16267006242283819), ((0, 6), 0.12542775034607853), ((0, 7), 0.03025399974428621), ((0, 8), 0.07043695757277932), ((0, 9), 0.015327410174732194), ((0, 10), 0.12276311143481825), ((0, 11), 0.1121934320757178), ((0, 12), 0.03952486400747672), ((0, 13), 0.052772373028530335), ((0, 14), 0.006571027700353911), ((0, 15), 0.17047809410031428), ((0, 16), 0.09966766630036666), ((0, 17), 0.1645381965477087), ((0, 18), 0.07976428030202641), ((0, 19), 0.1541710318926218), ((0, 20), 0.018808708003976052), ((0, 21), 0.16968910565971196), ((0, 22), 0.12654204642629974), ((0, 23), 0.0382244195393445), ((0, 24), 0.15804813969674278), ((0, 25), 0.12065854696831398), ((0, 26), 0.04919942900124078), ((0, 27), 0.15149008641710815), ((0, 28), 0.07311773238255924), ((0, 29), 0.14855178354736323)

True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

In [44]:
I = 2
J = 3
K = 4*10

small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

reference_solution = np.matmul(small_mat, big_mat)

def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())

  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)

  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER)

  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output) + 1
  K = max(k for ((i,k), vw) in reduce_output) + 1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

def INPUTFORMAT():
  first_mat = []

  for i in range(small_mat.shape[0]):
    for j in range(small_mat.shape[1]):
      first_mat.append(((0, i, j), small_mat[i,j]))

  global maps
  split_size = int(np.ceil(len(first_mat)/maps))

  for i in range(0, len(first_mat), split_size):
    yield first_mat[i:i+split_size]

  second_mat = []

  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      second_mat.append(((1, j, k), big_mat[j,k]))

  split_size = int(np.ceil(len(second_mat)/maps))

  for i in range(0, len(second_mat), split_size):
    yield second_mat[i:i+split_size]

def MAP_JOIN(k1, v1):
  (mat_num, i, j) = k1
  w = v1

  if mat_num == 0:
    yield (j, (mat_num, i, w))

  else:
    yield (i, (mat_num, j, w))

def REDUCE_JOIN(key, values):
  from_first_mat = [v for v in values if v[0] == 0]
  from_second_mat = [v for v in values if v[0] == 1]

  for f in from_first_mat:
    for s in from_second_mat:
      yield ((f[1], s[1]), f[2] * s[2])

def GET_JOINED():

  for j in joined:
    print("aa", j)
    yield j[1]

def MAP_MUL(k1, v1):
  yield (k1, v1)

def REDUCE_MUL(key, values):
  res_val = 0

  for v in values:
    res_val += v
  yield (key, res_val)

maps = 3
reducers = 2

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_JOIN, REDUCE_JOIN, COMBINER=None)
joined = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(joined)

mul_output = MapReduceDistributed(GET_JOINED, MAP_MUL, REDUCE_MUL, COMBINER=None)
pre_result = [(partition_id, list(partition)) for (partition_id, partition) in mul_output]
print(pre_result)

solution = []

for p in pre_result:

  for v in p[1]:
    solution.append(v)

print(solution)

np.allclose(reference_solution, asmatrix(solution))

126 key-value pairs were sent over a network.
[(0, [((0, 0), 0.20539858540185124), ((0, 1), 0.0009617393399619311), ((0, 2), 0.13047611452761615), ((0, 3), 0.20465822568661368), ((0, 4), 0.01055368971689809), ((0, 5), 0.06114878854464203), ((0, 6), 0.0910746412507865), ((0, 7), 0.20697739806196536), ((0, 8), 0.05756166084418511), ((0, 9), 0.1659228522647792), ((0, 10), 0.23072674234758836), ((0, 11), 0.01424618271184774), ((0, 12), 0.0021711610330737716), ((0, 13), 0.14653771710255284), ((0, 14), 0.03235767266988558), ((0, 15), 0.18128217934652152), ((0, 16), 0.027675105929419387), ((0, 17), 0.12734423628981112), ((0, 18), 0.18680461564369857), ((0, 19), 0.1389856980659263), ((0, 20), 0.1658731784909857), ((0, 21), 0.01813929559517629), ((0, 22), 0.14805560155936948), ((0, 23), 0.17557324879877406), ((0, 24), 0.11379728040406482), ((0, 25), 0.16260932966523542), ((0, 26), 0.12869766557493437), ((0, 27), 0.20369335319756918), ((0, 28), 0.09995589764048954), ((0, 29), 0.04218492514497515

True